<a href="https://colab.research.google.com/github/Tclack88/misc-analysis/blob/main/vietnamese_frequency_sentence_mining.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!wget "https://github.com/qhungngo/EVBCorpus/raw/master/EVBCorpus_EVBNews_v2.0.rar"

--2022-01-11 05:03:41--  https://github.com/qhungngo/EVBCorpus/raw/master/EVBCorpus_EVBNews_v2.0.rar
Resolving github.com (github.com)... 140.82.112.4
Connecting to github.com (github.com)|140.82.112.4|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/qhungngo/EVBCorpus/master/EVBCorpus_EVBNews_v2.0.rar [following]
--2022-01-11 05:03:42--  https://raw.githubusercontent.com/qhungngo/EVBCorpus/master/EVBCorpus_EVBNews_v2.0.rar
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 7310993 (7.0M) [application/octet-stream]
Saving to: ‘EVBCorpus_EVBNews_v2.0.rar’

EVBCorpus_EVBNews_v 100%[===================>]   6.97M  --.-KB/s    in 0.08s   

2022-01-11 05:03:42 (85.6 MB/s) - ‘EVBCorpus_EVBNews_v2.0.rar’ s

In [3]:
!unrar e "EVBCorpus_EVBNews_v2.0.rar"  &> /dev/null 

In [1]:
!pip install pandas==1.3.0
from pathlib import Path
import pandas as pd
pd.__version__

'1.3.0'

In [2]:
from bs4 import BeautifulSoup
import re

def process(instring): # remove punctuation and make lowercase
  outstring = re.sub(r'[^\w\s]', '', instring)
  return outstring.lower()  

sgml_files = Path('.').glob('*.sgml')
en = []
vn = []
for file in sgml_files:
  with open(file,'r') as f:
    doc = f.read()
    soup = BeautifulSoup(doc, 'html.parser')
    out = soup.find_all('spair')
    for i in range(len(out)):
      e = process(out[i].find('s', {'id':f'en{i+1}'}).getText())
      v = process(out[i].find('s', {'id':f'vn{i+1}'}).getText())
      if len(v.split()) <= 12:
        if v not in vn: # avoid duplicates
          en.append(e)
          vn.append(v)

In [3]:
print(len(vn), len (en))

9077 9077


In [4]:
freq = pd.read_html("https://1000mostcommonwords.com/1000-most-common-vietnamese-words/", header=0)[0]
freq[20:30]

,Number,Vietnamese,in English
20,21,nhưng,but
21,22,những gì,what
22,23,một số,some
23,24,là,is
24,25,nó,it
25,26,anh,you
26,27,hoặc,or
27,28,có,had
28,29,các,the
29,30,của,of


In [5]:
# TODO: unfinished attempt to split the words
import numpy as np
freq1 = freq.copy()
temp = freq1.Vietnamese.str.split(' ')
freq1 = freq1.reindex(freq1.index.repeat(temp.apply(len)))
freq1.Vietnamese = np.hstack(temp)
freq1.columns = ['number', 'vietnamese', 'english']
# freq1.Vietnamese = freq1.Vietnamese.str.split(" ")
freq1[20:30]

# data = data.reindex(data.index.repeat(temp.apply(len)))
# data['new_Seatblocks'] = np.hstack(temp)

,number,vietnamese,english
20,21,nhưng,but
21,22,những,what
21,22,gì,what
22,23,một,some
22,23,số,some
23,24,là,is
24,25,nó,it
25,26,anh,you
26,27,hoặc,or
27,28,có,had


In [6]:
freq_dict = dict(zip(freq1.vietnamese, freq1.number))

In [7]:
common_vn = []
common_en = []
for i, sentence in enumerate(vn):
  simple = 1
  for word in sentence.split():
    if word in list(freq1.vietnamese):
      pass
    else:
      simple = 0
  if simple:
    common_vn.append(sentence)
    common_en.append(en[i])

print(len(common_vn), len(common_en))

973 973


In [8]:
sentence_values = []
for sentence in common_vn:
  val = 0
  for word in sentence.split():
    word = word.lower()
    val += freq_dict[word]
  avg_val = val / len(sentence)
  sentence_values.append(avg_val)

In [9]:
len(sentence_values)

973

In [10]:
simple_df = pd.DataFrame({'sentence_values': sentence_values, 'common_vn':common_vn, 'common_en':common_en})
simple_df.head(20)

,sentence_values,common_vn,common_en
0,119.894737,lẽ ra phải như vậy,this is the way it should be
1,97.461538,cô bé chạy lên phòng mình,she ran up to her room
2,119.216216,cuối cùng cô bé quyết định bỏ nhà đi,finally she decided to run away
3,163.565217,kế hoạch sẽ thành công,the plan would work
4,143.321429,cô sẽ làm cho nó thành công,she would make it work
5,210.615385,con yêu mẹ,i love you
6,155.312500,đôi khi họ lại cười về việc này,sometimes they laughed about it
7,114.735294,cô bé thực hiện kế hoạch của mình,she has made her plan
8,194.000000,mẹ cũng yêu con mẹ cô nói,i love you too her mother said
9,126.031250,con biết điều đó mà phải không,you know that do nt you


In [18]:
simple_df = simple_df.sort_values('sentence_values').reset_index().drop(columns=['index'])

In [20]:
simple_df.to_excel('vietnamese_frequency.xlsx')